In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

load_dotenv(override=True)
hf_token=os.getenv('HF_TOKEN')

login(hf_token, add_to_git_credential=True)

code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
CODE_QWEN_URL = "https://h1vdol7jxhje3mpn.us-east-1.aws.endpoints.huggingface.cloud"

messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell me a joke."}
]

tokenizer = AutoTokenizer.pre_trained(code_qwen)
text = tokenizer.apply_chat_templates(messages, tokenize=False, add_generation_prompt=True)

client= InferenceClient(CODE_QWEN_URL, token=hf_token)
stream = client.chat(text, stream=True, details=True, max_new_tokens=3000)

result=""
for r in stream:
    print(r.token.text, end="")
